Please modify these SQuidASM programs to create another EPR pair using the quantum network processing unit:
```python

```

In [1]:
from typing import List

from netqasm.sdk.classical_communication.socket import Socket
from netqasm.sdk.connection import BaseNetQASMConnection
from netqasm.sdk.epr_socket import EPRSocket
from netqasm.sdk.qubit import Qubit

from squidasm.run.stack.config import GenericQDeviceConfig, LinkConfig, StackConfig, StackNetworkConfig
from squidasm.sim.stack.program import Program, ProgramContext, ProgramMeta
from squidasm.run.stack.run import run as run_simulation
from squidasm.util import create_two_node_network


name_of_network_node1: str = "node1"
name_of_network_node2: str = "node2"


class SquidASMProgram1(Program):
    NAME_OF_THIS_PROGRAM: str = "SquidASMProgram1"
    NAMES_OF_PEERS_WITH_CLASSICAL_SOCKET: List[str] = []
    NAMES_OF_PEERS_WITH_EPR_PAIR_SOCKET: List[str] = []
    MAXIMUM_NUMBER_OF_QUBITS_IN_THIS_PROGRAM: int = 2

    @property
    def meta(self) -> ProgramMeta:
        return ProgramMeta(
            name=self.NAME_OF_THIS_PROGRAM,
            csockets=self.NAMES_OF_PEERS_WITH_CLASSICAL_SOCKET,
            epr_sockets=self.NAMES_OF_PEERS_WITH_EPR_PAIR_SOCKET,
            max_qubits=self.MAXIMUM_NUMBER_OF_QUBITS_IN_THIS_PROGRAM,
        )

    def run(self, program_context: ProgramContext):
        connection_to_the_quantum_network_processing_unit: BaseNetQASMConnection = program_context.connection
        epr_socket_to_network_node2 = program_context.epr_sockets[name_of_network_node2]
        classical_socket_to_network_node2 = program_context.epr_sockets[name_of_network_node2]

        local_memory_qubit = Qubit(conn=connection_to_the_quantum_network_processing_unit)

        sent_epr_pairs = epr_socket_to_network_node2.create_keep(number=1)

        # send instructions to the the quantum network processing unit and await execution completes
        yield from connection_to_the_quantum_network_processing_unit.flush()

        result = {}
        return result


class SquidASMProgram2(Program):
    NAME_OF_THIS_PROGRAM: str = "SquidASMProgram2"
    NAMES_OF_PEERS_WITH_CLASSICAL_SOCKET: List[str] = ["node1"]
    NAMES_OF_PEERS_WITH_EPR_PAIR_SOCKET: List[str] = ["node1"]
    MAXIMUM_NUMBER_OF_QUBITS_IN_THIS_PROGRAM: int = 2

    @property
    def meta(self) -> ProgramMeta:
        return ProgramMeta(
            name=self.NAME_OF_THIS_PROGRAM,
            csockets=self.NAMES_OF_PEERS_WITH_CLASSICAL_SOCKET,
            epr_sockets=self.NAMES_OF_PEERS_WITH_EPR_PAIR_SOCKET,
            max_qubits=self.MAXIMUM_NUMBER_OF_QUBITS_IN_THIS_PROGRAM,
        )

    def run(self, program_context: ProgramContext):
        connection_to_the_quantum_network_processing_unit: BaseNetQASMConnection = program_context.connection

        epr_socket_to_network_node1 = program_context.epr_sockets[name_of_network_node1]
        classical_socket_to_network_node1 = program_context.epr_sockets[name_of_network_node1]

        local_memory_qubit = Qubit(conn=connection_to_the_quantum_network_processing_unit)

        received_epr_pairs = epr_socket_to_network_node1.recv(number=1)

        # send instructions to the the quantum network processing unit and await execution completes
        yield from connection_to_the_quantum_network_processing_unit.flush()

        result = {}
        return result


if __name__ == "__main__":
    instance_of_squidasm_program1 = SquidASMProgram1()
    instance_of_squidasm_program2 = SquidASMProgram2()

    run_simulation_this_number_of_times: int = 1

    run_simulation(
        config=create_two_node_network(
            node_names=[
                name_of_network_node1,
                name_of_network_node2,
            ],
            link_noise=0
        ),
        programs={
            name_of_network_node1: instance_of_squidasm_program1,
            name_of_network_node2: instance_of_squidasm_program2,
        },
        num_times=run_simulation_this_number_of_times,
    )